In [127]:
!pip install -U langchain-google-genai
!pip install --upgrade langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.59
    Uninstalling langchain-core-0.3.59:
      Successfully uninstalled langchain-core-0.3.59


In [ ]:
import langchain

In [128]:
Env_Keys = {
    "LANGSMITH_API_KEY" : "",
    "GOOGLE_API_KEY" : ""
}

In [90]:
os.environ["LANGSMITH_API_KEY"]

'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'

In [85]:
LangSmith_Config = {
    "LANGSMITH_TRACING" : "true",
    "LANGSMITH_ENDPOINT" : "https://api.smith.langchain.com",
    "LANGSMITH_PROJECT" : "pr-political-worth-69"
  }

In [86]:
import os
for (key, value) in Env_Keys.items():
  os.environ[key] = value
for (key, value) in LangSmith_Config.items():
  os.environ[key] = value

In [92]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_PROJECT"] = "pr-political-worth-69"

In [129]:
from typing import Optional

from pydantic import BaseModel, Field


class Person(BaseModel):

    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

In [130]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),

        ("human", "{text}"),
    ]
)

In [131]:
prompt_template

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are an expert extraction algorithm. Only extract relevant information from the text. If you do not know the value of an attribute asked to extract, return null for the attribute's value."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [136]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [134]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [137]:
structured_llm = llm.with_structured_output(schema=Person)

In [138]:
structured_llm

RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7b9d98165b10>, default_metadata=(), model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Person', 'description': '', 'parameters': {'properties': {'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'The name of the person'}, 'hair_color': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': "The color of the person's hair if known"}, 'height_in_meters': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'Height measured in meters'}}, 'type': 'object'}}}], 'ls_structured_output_format': {'kwargs': {'method': 'function_calling'}, 'schema': {'type': 'function', 'function': {'name': 'Person', 'description': '', 'parameters': {'properties'

In [140]:
text = "Alan Smith is 6 feet tall and has blond hair."
prompt = prompt_template.invoke({"text": text})
person = structured_llm.invoke(prompt)

In [141]:
person.name

'Alan Smith'